In [1]:
import pandas as pd
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

loo = LeaveOneOut()
# kf = KFold(n_splits=10, random_state=1000)

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_revisi = "F:/178/Tugas Akhir/dataset/revisi/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

# def crosval(clf,data,values):
#     scores = cross_val_score(clf,data,values,scoring='r2',cv=kf)
    
#     return scores

def crosval(clf,data,values):
    r2 = list()
    for train_index, test_index in loo.split(data):
        
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = values[train_index], values[test_index]
        trained = clf.fit(X_train,y_train)
        pred = clf.predict(X_test)
        trained.set_params(kernel=trained.kernel_)
        r2.append(r2_score(y_test, pred))
        
    return r2

In [4]:
kernels = 1.0 * RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-6, 1e6), 
                            alpha_bounds=(1e-6, 1e6))

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean,feature_range=(-1, 1))

(744, 26)


INITIAL MODEL

In [6]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False,  
                    random_state=1000) #init Arousal

CROSS VAL

In [7]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

AROUSAL

In [8]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.3910626   0.47459187  0.51632551  0.51301303  0.66232909  0.56969859
  0.51457676  0.59973048  0.64101183  0.52510323]
R2 Score Valence:	0.54 (+/- 0.08)


VALENCE

In [9]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.08964426  0.04189515  0.02299982 -0.16253443  0.35069548  0.08618049
  0.3195261   0.26035581  0.2604227   0.11276893]
R2 Score Valence:	0.14 (+/- 0.15)


# CASE 2

In [10]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 8)


INITIAL MODEL

In [11]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,
                    random_state=1000) #init Valence

CROSS VAL

In [12]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

AROUSAL

In [13]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.34577098  0.24491812  0.28127671  0.41154012  0.53725994  0.47689545
  0.51514342  0.55903965  0.32751106  0.44906844]
R2 Score Valence:	0.41 (+/- 0.10)


VALENCE

In [14]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.32255632  0.31200536  0.27817599  0.03938088  0.45160735  0.30297123
  0.39022827  0.3633554   0.12285081  0.05594909]
R2 Score Valence:	0.26 (+/- 0.13)


# CASE 3

In [15]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 3)


INITAL MODEL

In [16]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

CROSS VAL

In [17]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

AROUSAL

In [18]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.36172872  0.38060938  0.23187221  0.16438693  0.50029084  0.32493546
  0.33506575  0.36362014  0.48388714  0.49246438]
R2 Score Valence:	0.36 (+/- 0.10)


VALENCE

In [19]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.00952742  0.04575965 -0.01055374 -0.40370723  0.27498788  0.13892929
  0.07811918  0.11373871  0.27856919 -0.00287357]
R2 Score Valence:	0.05 (+/- 0.18)


# CASE 4

In [20]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 24)


INITIAL MODEL

In [21]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

CROSS VAL

In [22]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

AROUSAL

In [23]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.10354592  0.45910561  0.23705329  0.14424051  0.32759326  0.15533388
  0.25083502  0.13544426  0.26620285  0.36803702]
R2 Score Valence:	0.24 (+/- 0.11)


VALENCE

In [24]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [-0.22774086  0.01189957 -0.12254749 -0.20240256  0.1526967   0.13610621
  0.1010296  -0.12008601  0.09989383  0.12748756]
R2 Score Valence:	-0.00 (+/- 0.14)


# CASE 5

In [25]:
datafitur = "case5" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 34)


In [26]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

In [27]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [28]:
print("R2 Score Arousal:\t",scores_a)
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Arousal:	 [ 0.47788137  0.54026334  0.58890737  0.6361343   0.65247576  0.61614013
  0.63938133  0.65665649  0.59798816  0.58881866]
R2 Score Arousal:	0.60 (+/- 0.05)


In [29]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34238373  0.29560159  0.25651004  0.12817366  0.49427774  0.27253014
  0.46233635  0.34927316  0.32086149  0.1747295 ]
R2 Score Valence:	0.31 (+/- 0.11)


# CASE 6

In [30]:
datafitur = "case6" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape) 

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 29)


In [31]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

In [32]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [33]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.45541019  0.47778834  0.50730507  0.5127034   0.64907239  0.58841964
  0.53430288  0.58054331  0.61779314  0.55992374]
R2 Score Valence:	0.55 (+/- 0.06)


In [34]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.12556428  0.07701732  0.05260024 -0.14540139  0.36929763  0.15691064
  0.31013156  0.21442737  0.28470864  0.09442961]
R2 Score Valence:	0.15 (+/- 0.14)


# CASE 7

In [35]:
datafitur =  "case7" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 50)


In [36]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

In [37]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [38]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.37475968  0.53584835  0.57072896  0.43544335  0.66905966  0.61885491
  0.55388651  0.60272732  0.6683774   0.60761393]
R2 Score Valence:	0.56 (+/- 0.09)


In [39]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.07378561  0.15717188  0.02706023 -0.20362826  0.28292409  0.22447912
  0.29472578  0.15419424  0.24676829  0.23867419]
R2 Score Valence:	0.15 (+/- 0.14)


# CASE 8

In [40]:
datafitur =  "case8" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 37)


In [41]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [42]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [43]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.50399993  0.52917458  0.55526254  0.63147587  0.65819081  0.63151828
  0.64936933  0.6413492   0.59247438  0.60823464]
R2 Score Valence:	0.60 (+/- 0.05)


In [44]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.34752517  0.30763532  0.25344136  0.12935013  0.51098436  0.27091932
  0.45620458  0.33800073  0.31557591  0.17228014]
R2 Score Valence:	0.31 (+/- 0.11)


# CASE 9

In [45]:
datafitur =  "case9" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 58)


In [46]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [47]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [48]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.43734527  0.56481987  0.62743605  0.53644109  0.6553147   0.6589453
  0.63959572  0.66507667  0.6239438   0.64729954]
R2 Score Valence:	0.61 (+/- 0.07)


In [49]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.33282752  0.34042148  0.23223792  0.06299824  0.39626097  0.32132254
  0.4352018   0.31321752  0.28713712  0.3035634 ]
R2 Score Valence:	0.30 (+/- 0.10)


# CASE 10

In [50]:
datafitur =  "case10" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

(744, 61)


In [51]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [52]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

x = np.array(data)
ar = np.array(scaled_ar)
val = np.array(scaled_val)
scores_a = crosval(gpr_a,x,ar)
scores_v = crosval(gpr_v,x,val)

scores_a = np.array(scores_a)
scores_v = np.array(scores_v)

In [53]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

R2 Score Valence:	 [ 0.4468212   0.58337398  0.61119341  0.53480323  0.65842303  0.67015655
  0.6432077   0.66145445  0.62622194  0.66080131]
R2 Score Valence:	0.61 (+/- 0.07)


In [54]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Valence:	 [ 0.31889419  0.3503126   0.23094176  0.05290335  0.40348238  0.31129968
  0.42612237  0.29532972  0.27685798  0.2835904 ]
R2 Score Valence:	0.29 (+/- 0.10)
